In [2]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="/kaggle/input/multi-qwen-test/sft_data_multi_templates_gpt_text3_deepseek70_qwen4_train.jsonl")


In [3]:
# 加载 Qwen 模型和 tokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch
from transformers import TrainingArguments
from trl import SFTTrainer

model_name = "Qwen/Qwen2.5-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

2025-07-23 05:56:25.329878: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753250185.352381     787 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753250185.359328     787 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
# Tokenization
def tokenize_prompt(example):
    prompt = example["prompt"]
    label = example["label"]

    prompt_tokens = tokenizer(prompt, truncation=True, max_length=2048)
    label_tokens = tokenizer(label, truncation=True, max_length=2048)
    prompt_ids = prompt_tokens["input_ids"]
    label_ids = label_tokens["input_ids"]

    input_ids = prompt_ids + label_ids
    input_ids = input_ids[:4096]

    labels = [-100] * len(prompt_ids) + label_ids
    labels = labels[:4096]

    return {
        "input_ids": input_ids,
        "labels": labels,
        "attention_mask": [1] * len(input_ids)
    }
tokenized_dataset = dataset["train"].map(tokenize_prompt, remove_columns=dataset["train"].column_names)


# 加载模型 + 应用LoRA配置
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

model = prepare_model_for_kbit_training(model)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

training_args = TrainingArguments(
    output_dir="/kaggle/working/qwen_lora_output",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=1,
    num_train_epochs=5,
    logging_steps=5,
    save_steps=80,
    report_to="none",
    run_name="qwen_lora",
    learning_rate=2e-4,
    fp16=True,
)


In [ ]:
# 训练模型
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_args,
)

print("Number of training samples:", len(trainer.train_dataset))
print("Start training...")
trainer.train()
trainer.save_model()

[2025-07-23 05:56:53,849] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


[2025-07-23 05:56:55,296] [INFO] [logging.py:107:log_dist] [Rank -1] [TorchCheckpointEngine] Initialized with serialization = False


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Number of training samples: 246
Start training...


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:457: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we wi

Step,Training Loss
5,1.631800
10,1.750500
15,1.530000
20,1.562300
25,1.397800
30,1.351400
35,1.371900
40,1.317900
45,1.192000
50,1.249300


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

In [ ]:
import json
import re
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

# 加载 base 模型和 LoRA 权重
model_name = "Qwen/Qwen2.5-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

model = PeftModel.from_pretrained(base_model, "/kaggle/working/qwen_lora_output")
model.eval()

# 加载测试集
test_dataset = load_dataset("json", data_files="/kaggle/input/multi-qwen-test/sft_data_multi_templates_gpt_text3_deepseek70_qwen4_test.jsonl")["train"]

# label 提取函数
def extract_label(text: str) -> str:
    match = re.search(r"Label:\s*(Yes|No)", text, re.IGNORECASE)
    if match:
        return match.group(1).capitalize()
    return "Unknown"

# 推理与评估
correct = 0
total = 0

for sample in test_dataset:
    prompt = sample["prompt"]
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=2048).to("cuda:0")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=2048,
            do_sample=False,
            temperature=0.0,
            pad_token_id=tokenizer.pad_token_id
        )

    input_len = inputs["input_ids"].shape[1]
    gen_ids = outputs[0][input_len:]
    gen_text = tokenizer.decode(gen_ids, skip_special_tokens=True)
    pred_label = extract_label(gen_text)

    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("output_text", "---"*10, "\n", output_text)

    true_label = "Yes" if "Label: Yes" in sample["label"] else "No"

    total += 1
    if pred_label == true_label:
        correct += 1

    print(f"pred: {pred_label}, true: {true_label}")
    print(f"total: {total}, correct: {correct}")

# 准确率输出
accuracy = correct / total
print(f"\nAccuracy: {accuracy:.2%} ({correct}/{total})")